In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import scipy.sparse as sp

import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from time import time
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

from functions_HGPLS import train, test

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

model = AutoModelForMaskedLM.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
atoms = ["", "H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm"]
print(len(atoms))
ids = tokenizer(atoms).input_ids

toks = list()
for i in ids:
    if len(i) >= 3:
        toks.append(i[1])
    else:
        toks.append(i[0])

100


In [4]:
dataset, labels_g = dgl.load_graphs("../data/HIV_dgl_graphs")

In [5]:
labels_g = labels_g['glabel'].tolist()

In [6]:
# d = dataset[0].ndata["atomic"]
tok = torch.zeros((len(toks), 1))
for i in range(len(toks)):
    tok[i] = toks[i]

tok
transfo_atoms = model(tok.to(torch.long)).logits.reshape(100, -1)

pca = PCA(n_components=100, random_state=42)
res = pca.fit_transform(transfo_atoms.detach().numpy())

In [9]:
def create_features(g):
    degree = (g.out_degrees().float() + g.in_degrees().float()).numpy()
    num_edges = g.number_of_edges()
    num_nodes = g.number_of_nodes()
    nx_g = g.to_networkx().to_undirected()
    laplacian = nx.laplacian_matrix(nx_g).astype(np.float32).toarray()
    eigenvals, eigenvecs = np.linalg.eigh(laplacian)
    return [np.mean(degree), np.max(degree)/len(degree), 100*num_edges / (num_nodes * (num_nodes - 1))] + list(eigenvals[-3:])

In [6]:
dataset[0]

Graph(num_nodes=22, num_edges=46,
      ndata_schemes={'atomic': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={'type': Scheme(shape=(1,), dtype=torch.float32)})

In [11]:
new_dataset = list()
labels = list()
for i in tqdm(range(len(dataset))):
    g = dataset[i]
    l = labels_g[i]
    # features = list(np.mean(dataset.feature[g.ndata["_ID"]].numpy(), axis=0))
    # features = list(np.mean(g.ndata["attr"].numpy(), axis=0))
    # features = create_features(g)
    # new_dataset.append(features)
    ids=list(g.ndata["atomic"].numpy().flatten().astype(int))
    new_dataset.append(np.sum(res[ids], axis=0))
    labels.append(l)

100%|██████████| 2443/2443 [00:00<00:00, 58171.54it/s]


In [12]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(new_dataset, labels, test_size=0.25, random_state=42)

In [13]:
xgb =XGBClassifier(random_state=0)

xgb.fit(train_dataset, train_labels)
predxgb = xgb.predict(test_dataset)

In [14]:
accuracy_score(test_labels, predxgb)

0.5515548281505729

In [15]:
import sklearn

In [16]:
sklearn.metrics.confusion_matrix(test_labels, predxgb)

array([[149, 104,   8],
       [ 95, 133,  27],
       [ 12,  29,  54]], dtype=int64)